In [2]:

###################################################
# PROJE: Rating Product & Sorting Reviews in Amazon
###################################################

###################################################
# İş Problemi
###################################################

# E-ticaretteki en önemli problemlerden bir tanesi ürünlere satış sonrası verilen puanların doğru şekilde hesaplanmasıdır.
# Bu problemin çözümü e-ticaret sitesi için daha fazla müşteri memnuniyeti sağlamak, satıcılar için ürünün öne çıkması ve satın
# alanlar için sorunsuz bir alışveriş deneyimi demektir. Bir diğer problem ise ürünlere verilen yorumların doğru bir şekilde sıralanması
# olarak karşımıza çıkmaktadır. Yanıltıcı yorumların öne çıkması ürünün satışını doğrudan etkileyeceğinden dolayı hem maddi kayıp
# hem de müşteri kaybına neden olacaktır. Bu 2 temel problemin çözümünde e-ticaret sitesi ve satıcılar satışlarını arttırırken müşteriler
# ise satın alma yolculuğunu sorunsuz olarak tamamlayacaktır.

###################################################
# Veri Seti Hikayesi
###################################################

# Amazon ürün verilerini içeren bu veri seti ürün kategorileri ile çeşitli metadataları içermektedir.
# Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

# Değişkenler:
# reviewerID: Kullanıcı ID’si
# asin: Ürün ID’si
# reviewerName: Kullanıcı Adı
# helpful: Faydalı değerlendirme derecesi (up)
# reviewText: Değerlendirme
# overall: Ürün rating’i
# summary: Değerlendirme özeti
# unixReviewTime: Değerlendirme zamanı
# reviewTime: Değerlendirme zamanı Raw
# day_diff: Değerlendirmeden itibaren geçen gün sayısı
# helpful_yes: Değerlendirmenin faydalı bulunma sayısı
# total_vote: Değerlendirmeye verilen oy sayısı (toplam yorum sayısı



###################################################
# GÖREV 1: Average Rating'i Güncel Yorumlara Göre Hesaplayınız ve Var Olan Average Rating ile Kıyaslayınız.
###################################################

# Paylaşılan veri setinde kullanıcılar bir ürüne puanlar vermiş ve yorumlar yapmıştır.
# Bu görevde amacımız verilen puanları tarihe göre ağırlıklandırarak değerlendirmek.
# İlk ortalama puan ile elde edilecek tarihe göre ağırlıklı puanın karşılaştırılması gerekmektedir.

In [10]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [7]:
df = pd.read_csv(r"C:\Users\yenib\OneDrive\Masaüstü\amazon_review.csv")

In [9]:
df.head(4)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0


In [15]:
# Adım 1:   Ürünün ortalama puanını hesaplayınız
df["overall"].mean()

4.587589013224822

In [21]:
# Adım 2:  Tarihe göre ağırlıklı puan ortalamasını hesaplayınız.
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   int64         
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 460.9+ KB


In [29]:
df.sort_values(by="reviewTime", ascending=False).head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
2793,A2GTHHMSZUK5JP,B007WTAJTO,LaWanda,"[0, 0]",Great Price,5.0,16 GB,1405123200,2014-12-07,1,0,0
4364,A16V5PSWQI45Z7,B007WTAJTO,Strider,"[0, 0]",Best card for an 808 #26. Gives you 2.5 charge...,5.0,Best card for an 808 #26,1405123200,2014-12-07,1,0,0


In [31]:
current_date = pd.to_datetime('2014-12-08')

In [33]:
df["days"] = (current_date - df["reviewTime"]).dt.days

In [39]:
df["days"].head(5)

0    138
1    409
2    715
3    382
4    513
Name: days, dtype: int64

In [47]:
df.loc[df["days"] <= 30, "overall"].mean() * 28/100 + \
df.loc[(df["days"] > 30) & (df["days"] <= 90), "overall"].mean() * 26/100 + \
df.loc[(df["days"] > 90) & (df["days"] <= 180), "overall"].mean() * 24/100 + \
df.loc[(df["days"]  > 180), "overall"].mean() * 22/100 

4.6987161061560725

In [55]:
# Adım 3:  Ağırlıklandırılmışpuanlamada her bir zaman diliminin ortalamasını karşılaştırıp yorumlayınız
df.loc[df["days"] <= 30, "overall"].mean()  

4.742424242424242

In [57]:
df.loc[(df["days"] > 30) & (df["days"] <= 90), "overall"].mean() 

4.803149606299213

In [59]:
df.loc[(df["days"] > 90) & (df["days"] <= 180), "overall"].mean() 

4.649484536082475

In [61]:
df.loc[(df["days"]  > 180), "overall"].mean() 

4.573373327180434

In [79]:
# Görev 2:  Ürün için ürün detay sayfasında görüntülenecek 20 review’ibelirleyiniz.

In [83]:
# Adım 1:  helpful_no değişkenini üretiniz

In [65]:
df.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,138
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,409


In [71]:
df['helpful_no'] = df['total_vote'] - df['helpful_yes'] 

In [75]:
df['helpful_no'].unique()

array([  0,   1,   6,   2,   3,  10,  73,   8,   4,  68, 110, 183,  77,
        27, 126,  14,   9], dtype=int64)

In [85]:
# Adım 2:  score_pos_neg_diff, score_average_ratingve wilson_lower_boundskorlarını hesaplayıp veriye ekleyiniz.

In [87]:
df['score_pos_neg_diff'] = df['helpful_yes'] - df['helpful_no'] 

In [119]:
import math
import scipy.stats as st
import pandas as pd

# Fonksiyonu tanımlayalım
def score_confidence_interval(row, confidence=0.95):
    up = row['helpful_yes']
    down = row['helpful_no']
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

# apply fonksiyonu ile her satıra uygulayalım
df['wilson_lower_bound'] = df.apply(score_confidence_interval, axis=1)


In [135]:
df.head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,138,0,0,0.0,0.000000
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,409,0,0,0.0,0.000000
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,715,0,0,0.0,0.000000
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,382,0,0,0.0,0.000000
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,513,0,0,0.0,0.000000
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.0,Not a lot to really be said,1367193600,2013-04-29,588,0,0,588,0,0,0.0,0.000000
6,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,"[0, 0]",I have it in my phone and it never skips a bea...,5.0,Works well,1382140800,2013-10-19,415,0,0,415,0,0,0.0,0.000000
7,A3CW0ZLUO5X2B1,B007WTAJTO,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",It's hard to believe how affordable digital ha...,5.0,32 GB for less than two sawbucks...what's not ...,1404950400,2014-10-07,62,0,0,62,0,0,0.0,0.000000
8,A2CYJO155QP33S,B007WTAJTO,4evryoung,"[1, 1]",Works in a HTC Rezound. Was running short of ...,5.0,Loads of room,1395619200,2014-03-24,259,1,1,259,0,1,0.2,0.206549
9,A2S7XG3ZC4VGOQ,B007WTAJTO,53rdcard,"[0, 0]","in my galaxy s4, super fast card, and am total...",5.0,works great,1381449600,2013-11-10,393,0,0,393,0,0,0.0,0.000000
